# RAG Vector Visualization

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("StudentRule.pdf") # Use your own pdf.

/opt/anaconda3/envs/rag_vector_visualization/lib/python3.12/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [3]:
docs = loader.load()

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [6]:
from langchain import hub

retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

/opt/anaconda3/envs/rag_vector_visualization/lib/python3.12/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
rag_chain.invoke("세명컴퓨터고등학교 교칙 중 전자기기(휴대전화)는 무엇이 허용되고 무엇이 금지되어있는지 알려주세요.")

'세명컴퓨터고등학교에서는 전자기기, 즉 휴대전화를 학교 등교 시 소지하는 것은 허용되지만, 수업 시간에는 전원을 끄고 시험 시간에는 소지할 수 없습니다. 또한, 휴대전화를 이용한 동영상 촬영은 금지되어 있습니다. 규정을 위반할 경우 주의 조치를 받으며, 반복 위반 시 분리 보관 및 벌점이 부여될 수 있습니다.'

In [10]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

pca = PCA(n_components=3)
tsne = TSNE(n_components=3)
umap = umap.UMAP(n_components=3)

/opt/anaconda3/envs/rag_vector_visualization/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
vectors = vectorstore.get(include=['embeddings'])['embeddings']

In [12]:
pca_vectors = pca.fit_transform(vectors)
tsne_vectors = tsne.fit_transform(vectors)
umap_vectors = umap.fit_transform(vectors)

In [17]:
import plotly.express as px

pca_fig = px.scatter_3d(
    x=pca_vectors[:, 0],
    y=pca_vectors[:, 1],
    z=pca_vectors[:, 2],
    title='PCA Visualization',
    labels={'x': 'PCA 1', 'y': 'PCA 2'}
)

tsne_fig = px.scatter_3d(
    x=tsne_vectors[:, 0],
    y=tsne_vectors[:, 1],
    z=tsne_vectors[:, 2],
    title='t-SNE Visualization',
    labels={'x': 't-SNE 1', 'y': 't-SNE 2'}
)

umap_fig = px.scatter_3d(
    x=umap_vectors[:, 0],
    y=umap_vectors[:, 1],
    z=umap_vectors[:, 2],
    title='UMAP Visualization',
    labels={'x': 'UMAP 1', 'y': 'UMAP 2'}
)

pca_fig.show()
tsne_fig.show()
umap_fig.show()